In [106]:
import pandas as pd
import numpy as np
from os import path
import gtfparse
import pyensembl
pyensembl.EnsemblRelease(release=87)
import gspread
from  oauth2client.service_account  import ServiceAccountCredentials
import gspread
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html
Go to the Google APIs Console.
Create a new project.
Click Enable API. Search for and enable the Google Drive API.
Create credentials for a Web Server to access Application Data.
Name the service account and grant it a Project Role of Editor.
Download the JSON file.
Copy the JSON file to your code directory
Open JSON and find "client_email"
share google sheet with this email

In [145]:
scope = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
# Path to .json
path = '/bigstore/GeneralStorage/Zach/MERFISH/Probe_Design/ProbeDesign-92ebdb6a17a7.json'
creds = ServiceAccountCredentials.from_json_keyfile_name(path, scope)
client = gspread.authorize(creds)
# Start with a gene list 
expression_worksheet = client.open("Housekeeping_Gene_List").sheet1
expression = expression_worksheet.get_all_records()
expression_df = pd.DataFrame(expression)

INFO:oauth2client.client:Refreshing access_token


In [108]:
expression_df

,Barcode,Description,FPKM,Gene,Gene_ID,Isoform %,Length,Oligos,Transcript_ID
0,,phosphoglycerate kinase 1 [Source:MGI Symbol;A...,7.191000e+01,Pgk1,ENSMUSG00000062070,,1825,,ENSMUST00000081593
1,,"succinate dehydrogenase complex, subunit A, fl...",5.106000e+01,Sdha,ENSMUSG00000021577,,2900,,ENSMUST00000022062
2,,TATA box binding protein [Source:MGI Symbol;Ac...,9.768260e-10,Tbp,ENSMUSG00000014767,,2067,,ENSMUST00000118001
3,,"H2A histone family, member Z [Source:MGI Symbo...",2.683060e+01,H2afz,ENSMUSG00000037894,,810,,ENSMUST00000174561
4,,glyceraldehyde-3-phosphate dehydrogenase [Sour...,0.000000e+00,Gapdh,ENSMUSG00000057666,,1273,,ENSMUST00000117757
5,,peptidylprolyl isomerase A [Source:MGI Symbol;...,1.121390e-05,Ppia,ENSMUSG00000071866,,1450,,ENSMUST00000213200
6,,"glucuronidase, beta [Source:MGI Symbol;Acc:MGI...",3.017270e+00,Gusb,ENSMUSG00000025534,,2038,,ENSMUST00000111308
7,,heat shock protein 1B [Source:MGI Symbol;Acc:M...,1.166990e-01,Hsp70,ENSMUSG00000090877,,2803,,ENSMUST00000172753
8,,tyrosine 3-monooxygenase/tryptophan 5-monooxyg...,1.924170e+01,Ywhaz,ENSMUSG00000022285,,2122,,ENSMUST00000110362
9,,"actin, beta [Source:MGI Symbol;Acc:MGI:87904]",2.575700e+02,Actb,ENSMUSG00000029580,,1242,,ENSMUST00000167721


In [59]:
# removing trnascript versions from worksheet
# Select a range
Transcript_ID = expression_worksheet.range('G2:G189')
# Edit range
for cell in Transcript_ID:
    cell.value = cell.value.split('.')[0]
# Update in batch
expression_worksheet.update_cells(Transcript_ID)

{'spreadsheetId': '11MMvEY4s8oUIGf5kyoA4QOKs-Br2myyNooNmSO7-nEU',
 'updatedCells': 188,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!G2:G189',
 'updatedRows': 188}

In [9]:
def col_cells(worksheet, col):
    """Returns a range of cells in a `worksheet`'s column `col`."""
    start_cell = worksheet.get_addr_int(1, col)
    end_cell = worksheet.get_addr_int(worksheet.row_count, col)

    return worksheet.range('%s:%s' % (start_cell, end_cell))

In [10]:
def read_codebook(cbook_fname):
    """Read CSV of n-bit codewords."""
    cwords = []
    with open(cbook_fname, 'r') as f:
        column_name = f.readline().strip()
        for l in f.readlines():
            cwords.append(l.strip())
    return cwords
codewords = read_codebook('/home/zach/Documents/Untitled Folder/cbook_140MHD4_200MHD2.txt') # need to fill in
np.random.shuffle(codewords)
print('Number of codewords in codebook -', len(codewords))

FileNotFoundError: [Errno 2] No such file or directory: '/home/zach/Documents/Untitled Folder/cbook_140MHD4_200MHD2.txt'

Need to download biomart file for the species of interest
This can be done here http://uswest.ensembl.org/biomart/martview/121d069e69616b12e28fa129a1e20f1c 

In [5]:
def geneSymbol_to_ensembl(gene_symbol_list, biomart_download_fname,
                          organism='mouse', min_length=1000):
    """
    Look up the ensembl of gene symbols.
    """
    annotations = []
    with open(biomart_download_fname, 'r') as f:
        transcript_df = pd.read_csv(f)
        #genes = [i.value for i in gene_symbol_list if i != '']
        genes = gene_symbol_list
        transcript_df = transcript_df[transcript_df[u'Gene name'].isin(genes)]
    for cell in gene_symbol_list:
        gene = cell#.value
        if gene == '':
            continue

        transcripts = transcript_df[transcript_df[u'Gene name'] == gene].drop_duplicates('Transcript stable ID')
        transcripts = transcripts[transcripts[u'Transcript type']==u'protein_coding'].sort_values('Transcript length (including UTRs and CDS)', ascending=False)
        
        if len(transcripts)==0:
            print('Failed finding: ', gene)
            annotations.append((gene, None))
        elif transcripts.iloc[0]['Transcript length (including UTRs and CDS)'] > min_length:
            annotations.append((gene, transcripts))
        else:
            print('Gene too short: ', gene)
            annotations.append((gene, transcripts))
    return annotations
#

In [61]:

def ensymbl_to_geneinfo(Gene_Stable_IDs, biomart_download_fname_path,
                          organism='mouse', min_length=1000):
    annotations = []
    with open(biomart_download_fname_path, 'r') as f:
        transcript_df = pd.read_csv(f)
        genes = [i for i in Gene_Stable_IDs if i != '']
        transcript_df = transcript_df[transcript_df[u'Gene stable ID'].isin(genes)]
    for cell in Gene_Stable_IDs:
        gene = cell#.value
        if gene == '':
            continue
        transcripts = transcript_df[transcript_df[u'Gene stable ID'] == gene].drop_duplicates('Transcript stable ID')
        transcripts = transcripts[transcripts[u'Transcript type']==u'protein_coding'].sort_values('Transcript length (including UTRs and CDS)', ascending=True)
        if len(transcripts)==0:
            print('Failed finding: ', gene)
            annotations.append((gene, None))
        elif transcripts.iloc[0]['Transcript length (including UTRs and CDS)'] > min_length:
            annotations.append((gene, transcripts))
        else:
            print('Gene too short: ', gene)
            annotations.append((gene, transcripts))
    return annotations


In [56]:

def ensymbl_to_geneinfo(Gene_Stable_IDs, biomart_download_fname_path,expression_path,
                          organism='mouse', min_length=1000):
    annotations = []
    FPKM_df = pd.read_csv(open(expression_path, 'r'))
    transcript_df = pd.read_csv(open(biomart_download_fname_path, 'r'))
    genes = [i for i in Gene_Stable_IDs if i != '']
    FPKM_df = FPKM_df[FPKM_df[u'gene_id'].isin(genes)]
    for cell in Gene_Stable_IDs:
        gene = cell#.value
        if gene == '':
            continue
        gene_FPKM_df = FPKM_df[FPKM_df[u'gene_id'] == gene].drop_duplicates('transcript_id')
        gene_FPKM_df = gene_FPKM_df[gene_FPKM_df[u'FPKM']>0].sort_values('FPKM', ascending=False)
        if len(gene_FPKM_df)==0:
            print('Failed finding: ', gene)
            annotations.append((gene, None))
        else:
            tids = [i for i in gene_FPKM_df['transcript_id'] if i != '']
            transcripts = transcript_df[transcript_df[u'Transcript stable ID'].isin(tids)]
            expression = []
            for tid in transcripts['Transcript stable ID']:
                expression.append(gene_FPKM_df[gene_FPKM_df['transcript_id']==tid]['FPKM'].iloc[0])
            transcripts['FPKM']=expression
            transcripts = transcripts.sort_values('FPKM', ascending=False)
            annotations.append((gene, transcripts))
    return annotations


In [137]:
len(genes[50])

0

In [140]:
biomart_download_fname = '/bigstore/binfo/mouse/mouse_ensemble.txt'
expression_path = '/bigstore/GeneralStorage/Zach/MERFISH/Probe_Design/MERFISH_analysis-master/mouse/Zach/Cornea/Expression/no_versions_isoforms_tracking.csv'
FPKM_df = pd.read_csv(open(expression_path, 'r'))
transcript_df = pd.read_csv(open(biomart_download_fname, 'r'))

gids = expression_worksheet.range('F2:F3000')
tids = expression_worksheet.range('G2:G3000')
fpkm = expression_worksheet.range('C2:C3000')
length = expression_worksheet.range('D2:D3000')
descript = expression_worksheet.range('B2:B3000')
gene_name = expression_worksheet.range('A2:A3000')

genes = [i.value for i in gids if len(i.value)>0]
FPKM_df = FPKM_df[FPKM_df[u'gene_id'].isin(genes)]
transcript_df = transcript_df[transcript_df[u'Gene stable ID'].isin(genes)]
for i,gene in enumerate(genes):
    gene = gene
    gene_fpkm_df = FPKM_df[(FPKM_df['gene_id']==gene)&(FPKM_df['length']>500)].sort_values('FPKM', ascending=False)
    tid = gene_fpkm_df['transcript_id'].iloc[0]
    tids[i].value = tid
    fpkm[i].value = str(gene_fpkm_df[gene_fpkm_df['transcript_id']==tid]['FPKM'].iloc[0])
    length[i].value = str(gene_fpkm_df[gene_fpkm_df['transcript_id']==tid]['length'].iloc[0])
    annotations = transcript_df[transcript_df[u'Transcript stable ID']==tid]
    descript[i].value = annotations['Gene description'].iloc[0]
    gene_name[i].value = annotations['Gene name'].iloc[0]
expression_worksheet.update_cells(tids)
expression_worksheet.update_cells(fpkm)
expression_worksheet.update_cells(length)
expression_worksheet.update_cells(descript)
expression_worksheet.update_cells(gene_name)


{'spreadsheetId': '1TmiXNh_yku0GHz6Jw76f0VPZH_h87kqJrrUUtJPfahU',
 'updatedRange': 'Sheet1!A2:A3000',
 'updatedRows': 2999,
 'updatedColumns': 1,
 'updatedCells': 2999}

In [57]:
# Add info to spreadsheet
# WARNING - you must figure out column numbers and change bits for new gene sets
biomart_download_fname = '/bigstore/binfo/mouse/mouse_ensemble.txt'
expression_path = '/bigstore/GeneralStorage/Zach/MERFISH/Probe_Design/MERFISH_analysis-master/mouse/Zach/Cornea/Expression/no_versions_isoforms_tracking.csv'

gene_symbol_list = expression_df['Gene_ID']
gene_annotations = ensymbl_to_geneinfo(gene_symbol_list, biomart_download_fname,expression_path, organism='finch', min_length=1000)
#gname = expression_worksheet.range('F2:F3000')
tname = expression_worksheet.range('G2:G3000')
length = expression_worksheet.range('D2:D3000')
descript = expression_worksheet.range('B2:B3000')
fpkm = expression_worksheet.range('C2:C3000')
#gene = expression_worksheet.range('A2:A3000')
loc = 0
update = []
for idx, g in enumerate(gene_annotations):
    info = g[1]
    if info is None:
        continue
    #Need to better select transcripts here
    max_transcript = info['Transcript stable ID'].iloc[loc]
    max_length = info['Transcript length (including UTRs and CDS)'].iloc[loc]
    descript_val = info['Gene description'].iloc[loc]
    fpkm_val = info['FPKM'].iloc[loc]
    #gene_name = info['Gene name'].iloc[loc]
    #gene_id = info['Gene stable ID'].iloc[loc]
    tname[idx].value = max_transcript
    length[idx].value = str(max_length)
    descript[idx].value = str(descript_val)
    fpkm[idx].value = str(fpkm_val)
    #gene[idx].value = str(gene_name)
    #gname[idx].value = str(gene_id)
expression_worksheet.update_cells(tname)
expression_worksheet.update_cells(length)
expression_worksheet.update_cells(descript)
expression_worksheet.update_cells(fpkm)
#expression_worksheet.update_cells(gene)
#expression_worksheet.update_cells(gname)

/home/zach/miniconda3/envs/pyspots/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'spreadsheetId': '1TmiXNh_yku0GHz6Jw76f0VPZH_h87kqJrrUUtJPfahU',
 'updatedRange': 'Sheet1!C2:C3000',
 'updatedRows': 2999,
 'updatedColumns': 1,
 'updatedCells': 2999}

In [37]:
def Trascriptome_append(transcript_IDs,file):
    annotations = []
    with open(file, 'r') as f:
        transcriptome_df = pd.read_csv(f)
        genes = [i for i in transcript_IDs if i != '']
        transcriptome_df = transcriptome_df[transcriptome_df[u'transcript_id'].isin(genes)] 
    for cell in transcript_IDs:
        gene = cell
        if gene == '':
            continue
        transcripts = transcriptome_df[transcriptome_df[u'transcript_id'] == gene]
        if len(transcripts)==0:
            print('Failed finding: ', gene)
            annotations.append((gene, None))
        else:
            annotations.append((gene, transcripts))
    return annotations 

In [44]:
file = '/bigstore/GeneralStorage/Zach/MERFISH/Probe_Design/MERFISH_analysis-master/mouse/Zach/Cornea/Expression/no_versions_isoforms_tracking.csv'
transcriptome_df = pd.read_csv(open(file, 'r'))
transcriptome_df.sort_values('FPKM', ascending=True)

,transcript_id,class_code,nearest_ref_id,gene_id,gene_short_name,tss_id,locus,length,coverage,FPKM,FPKM_conf_lo,FPKM_conf_hi,FPKM_status
121192,ENSMUST00000086878,-,-,ENSMUSG00000024172,St6gal2,-,chr17:55445388-55499226,2606,0.00,0.00,0.000,0.000000,OK
50083,ENSMUST00000203475,-,-,ENSMUSG00000108147,RP24-446A8.1,-,chr6:139575923-139577205,1282,0.00,0.00,0.000,0.000000,OK
50084,ENSMUST00000170650,-,-,ENSMUSG00000092164,Rergl,-,chr6:139492972-139501976,958,0.00,0.00,0.000,0.000000,OK
50085,ENSMUST00000203246,-,-,ENSMUSG00000107679,RP24-446A8.2,-,chr6:139555558-139557611,498,0.00,0.00,0.000,0.000000,OK
50086,ENSMUST00000083948,-,-,ENSMUSG00000065882,n-R5s168,-,chr6:140319629-140319733,104,0.00,0.00,0.000,0.000000,OK
50087,ENSMUST00000203859,-,-,ENSMUSG00000108229,RP23-330J17.2,-,chr6:140309070-140309740,670,0.00,0.00,0.000,0.000000,OK
50088,ENSMUST00000203881,-,-,ENSMUSG00000108047,RP23-78J15.7,-,chr6:140069993-140070498,505,0.00,0.00,0.000,0.000000,OK
50089,ENSMUST00000204065,-,-,ENSMUSG00000107407,RP23-78J15.4,-,chr6:140132066-140132508,442,0.00,0.00,0.000,0.000000,OK
50090,ENSMUST00000204425,-,-,ENSMUSG00000107572,RP23-330J17.1,-,chr6:140295462-140296456,994,0.00,0.00,0.000,0.000000,OK
50091,ENSMUST00000158933,-,-,ENSMUSG00000089558,Gm24174,-,chr6:140166534-140166645,111,0.00,0.00,0.000,0.000000,OK


In [38]:
# Add FPKM data
transcript_IDs = expression_df['Transcript_ID']
file = '/bigstore/GeneralStorage/Zach/MERFISH/Probe_Design/MERFISH_analysis-master/mouse/Zach/Cornea/Expression/no_versions_isoforms_tracking.csv'
Transcriptome_annotations = Trascriptome_append(transcript_IDs,file)
FPKM = expression_worksheet.range('C2:C189')
for idx, g in enumerate(Transcriptome_annotations):
    info = g[1]
    if info is None:
        continue
    fpkm = info['FPKM'].iloc[0]
    FPKM[idx].value = fpkm
expression_worksheet.update_cells(FPKM)

{'spreadsheetId': '1TmiXNh_yku0GHz6Jw76f0VPZH_h87kqJrrUUtJPfahU',
 'updatedRange': 'Sheet1!C2:C189',
 'updatedRows': 188,
 'updatedColumns': 1,
 'updatedCells': 188}

In [90]:
# Add info to spreadsheet
# WARNING - you must figure out column numbers and change bits for new gene sets
gene_annotations = ensymbl_to_geneinfo(expression_worksheet.range('F2:F205'), '/home/zach/Documents/Untitled Folder/mouse_gene_info2.txt', organism='mouse')
gname = expression_worksheet.range('F2:F189')
tname = expression_worksheet.range('G2:G189')
length = expression_worksheet.range('D2:D189')
descript = expression_worksheet.range('B2:B189')
gene = expression_worksheet.range('A2:A189')
update = []
for idx, g in enumerate(gene_annotations):
    info = g[1]
    if info is None:
        continue
    max_transcript = info['Transcript stable ID'].iloc[0]
    max_length = info['Transcript length (including UTRs and CDS)'].iloc[0]
    descript_val = info['Gene description'].iloc[0]
    gene_name = info['Gene name'].iloc[0]
    tname[idx].value = max_transcript
    length[idx].value = str(max_length)
    descript[idx].value = str(descript_val)
    gene[idx].value = str(gene_name)
expression_worksheet.update_cells(tname)
expression_worksheet.update_cells(length)
expression_worksheet.update_cells(descript)
expression_worksheet.update_cells(gene)

Gene too short:  ENSMUSG00000046259
Gene too short:  ENSMUSG00000048455
Gene too short:  ENSMUSG00000056054
Gene too short:  ENSMUSG00000044303
Gene too short:  ENSMUSG00000049775
Gene too short:  ENSMUSG00000001131
Gene too short:  ENSMUSG00000056071


{'spreadsheetId': '11MMvEY4s8oUIGf5kyoA4QOKs-Br2myyNooNmSO7-nEU',
 'updatedCells': 188,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!A2:A189',
 'updatedRows': 188}

In [146]:
readout_names = ['RS0095', 'RS0109', 'RS0175', 'RS0237', 'RS0307', 'RS0332', 'RS0384', 'RS0406', 
                'RS0451', 'RS0468', 'RS0548', 'RS64.0', 'RS156.0', 'RS278.0', 'RS313.0', 'RS643.0', 
                'RS740.0', 'RS810.0']
def write_codebook(rows, fname, readout_names, codebook_style = '148MHD4'):
    with open(fname, 'w') as f:
        f.write('version'+','+str(1)+'\n')
        f.write('codebook_name'+','+codebook_style+'\n')
        f.write('bit_names,'+','.join(readout_names)+'\n')
        f.write('name, id, barcode\n')
        for row in rows:
            f.write(','.join([row[0], row[1], row[2]+'\n']))

In [147]:
# Generate Codebook
row_tuples = []

for idx, row in expression_df.drop_duplicates('Transcript_ID').iterrows():
    row_tuples.append((row['Gene'], row['Transcript_ID'].split('.')[0], str(row['Barcode'])))

write_codebook(row_tuples, '/bigstore/GeneralStorage/Zach/MERFISH/Probe_Design/results/Housekeeping/Housekeeping_Codebook.txt', readout_names)
row_tuples

[('Pgk1', 'ENSMUST00000081593', '1000110010000000'),
 ('Sdha', 'ENSMUST00000022062', '1000110010000000'),
 ('Tbp', 'ENSMUST00000118001', '1000110010000000'),
 ('H2afz', 'ENSMUST00000174561', '1000110010000000'),
 ('Gapdh', 'ENSMUST00000147954', '1000110010000000'),
 ('Ppia', 'ENSMUST00000090749', '1000110010000000'),
 ('Gusb', 'ENSMUST00000026613', '1000110010000000'),
 ('Hspa1b', 'ENSMUST00000172753', '1000110010000000'),
 ('Ywhaz', 'ENSMUST00000022894', '1000110010000000'),
 ('Actb', 'ENSMUST00000100497', '1000110010000000'),
 ('B2m', 'ENSMUST00000102476', '1000110010000000'),
 ('Hk2', 'ENSMUST00000168725', '1000110010000000'),
 ('Hprt', 'ENSMUST00000026723', '1000110010000000'),
 ('Arf1', 'ENSMUST00000061242', '1000110010000000'),
 ('Cox7a2l', 'ENSMUST00000025095', '1000110010000000'),
 ('D8Ertd738e', 'ENSMUST00000019506', '1000110010000000'),
 ('Eif1', 'ENSMUST00000049385', '1000110010000000'),
 ('Eif4g2', 'ENSMUST00000161051', '1000110010000000'),
 ('Eif5a', 'ENSMUST00000108608', 

Now design probes
parse_merfish_oligos to add results to spreadsheet
Filter genes with low probes and repeat
Decide on final gene list and run agian


In [124]:
def parse_merfish_oligos(fname, counts_df = None,
                         counts_df_column='FPKM', tid_column='transcript_id'):
    """
    Bit hacky - should refactor and figure out how to handle missing isoform specificity info.
    """
    from Bio import SeqIO
    import pandas as pd
    df = pd.DataFrame(columns=['experiemnt', 'pleft', 'ro1', 'ro2', 'ro3', 
                               'pright', 'seq', 'gene', 'tid', 
                              'start', 'length', 'gc', 'tm', 'specicity'])
    readout_dict = {}
    oligos = SeqIO.FastaIO.SimpleFastaParser(open(fname, 'r'))
    rows = []
    fpkms = []
    for header, seq in oligos:
        fields = header.split(' ')
        experiment = str(fields[0])
        primer_left = str(fields[1])
        primer_seqL = seq[:20]
        primer_seqR = seq[-20:]
        readout1 = str(fields[2])
        readout_dict[readout1] = seq[20+1:20+1+20]
        isoSpecificity = 1
        # Order is different if Readouts are RO1/RO2 - encoding - RO3 vs RO1 - encoding - RO2/RO3
        # Check with if statement and handle accordingly.
        if '__' not in fields[3]:
            readout2 = str(fields[3])
            encoding = str(fields[4])
            readout3 = fields[5]
            ro2_start_idx = 41
            ro3_start_idx = 92
            readout_dict[readout3] = seq[20+20+20+1+30+1:20+1+20+20+30+1+20]
            readout_dict[readout2] = seq[20+1+20:20+1+20+20]
            primer_right = fields[6]
            gene, tid, start, length, gc, tm, specificity  = encoding.split('__')


#             gene, tid, start, length, gc, tm, specificity, isoSpecificity  = encoding.split('__')
            encoding_region = seq[20+1+20+20:20+20+1+20+30]
        else:
            encoding = fields[3]
#             isoSpecificity = fields[4]
            readout2 = fields[4]
            readout3 = fields[5]
            ro2_start_idx = 72
            ro3_start_idx = 92
            readout_dict[readout2] = seq[20+2+20+30:20+20+30+20+2]
            readout_dict[readout3] = seq[20+2+20+30+20:20+20+30+20+20+2]
            primer_right = fields[6]
            gene, tid, start, length, gc, tm, specificity  = encoding.split('__')
#             gene, tid, start, length, gc, tm, specificity, isoSpecificity  = encoding.split('__')
        # IMPLEMENT READOUT DICT In If Else
            encoding_region = seq[20+1+20:20+1+20+30]
        
        rows.append([experiment, primer_seqL, readout1, readout2, readout3, 
                       primer_seqR, encoding_region, seq, gene, tid, start,
                     length, gc, tm, specificity, isoSpecificity, header])
    df = pd.DataFrame(rows, columns=['experiment', 'pleft', 'ro1', 'ro2', 'ro3', 
                               'pright', 'encodingRegion', 'seq', 'gene', 'tid', 
                              'start', 'length', 'gc', 'tm', 'specificity', 'isoSpecificity', 'header'])
    df = df.drop_duplicates(subset=['gene', 'encodingRegion'])
#     if isinstance(counts_df, pd.DataFrame):
#         for tid in df.tid.unique():
#             fpkm = counts_df[counts_df[tid_column]==tid][counts_df_column]
#             tid_idx = df[df.tid==tid].index
#             for i in tid_idx:
#                 df.set_value(i, counts_df_column, fpkm.values[0])
#     df = df.convert_objects(convert_numeric=True)
#     df.sort_values(['gene', 'specificity', 'isoSpecificity'], ascending=False, inplace=True)
#     df['iso_off_spots'] = (df[counts_df_column] - df['isoSpecificity']*df[counts_df_column])/df['isoSpecificity']
#     df['gene_off_spots'] = (df[counts_df_column] - df['specificity']*df[counts_df_column])/df['specificity']
#     df = df.drop_duplicates('tid')
    return df, primer_seqL, primer_seqR, readout_dict

def trim_oligos_to_fit(oligo_df, multi_transcripts_cutoff = 148, min_oligos=48):
    df2 = oligo_df.copy()
    c = Counter(df2.gene)
    high_count = {}
    for g, count in c.items():
        if count < min_oligos:
            print(g, count)
#             c.pop(g)
            if g not in ['SNAI2', 'SNAI1', 'ORAI1', 'P2RY11', 'INPP1', 'ACTA2', 'PICK1']:
                df2.drop(df2[df2.gene==g].index, inplace=True)
#         if count>multi_transcripts_cutoff:
#             high_count[g] = count
#             ixes = list(df2[df2.gene==g].index)
#             ixes = np.random.choice(ixes, size=multi_transcripts_cutoff, replace=False)
#             df2.drop(ixes, inplace=True)
    return df2

# def balance_readouts(df, per_tid=64, fa_out='mergos.fa'):
#     from itertools import repeat
#     tids = df.groupby(group)
#     f = open(fa_out, 'w')
#     new_df = pd.DataFrame()
#     counters = []
#     for name, group in tids:
#         r_used = pd.unique(np.concatenate((group.ro1.unique(),group.ro2.unique(),group.ro3.unique())))
        
def balance_readouts(df, primersL, primersR, readouts, per_tid=64, group='tid',
                     fa_out='python_mergos.fa', sep='__'):
    verbose=False
    from itertools import repeat
    tids = df.groupby(group)
    f = open(fa_out, 'w')
    new_df = pd.DataFrame()
    counters = {}
    for name, group in tids:
        counts = Counter()
        r_used = pd.unique(np.concatenate((group.ro1.unique(),group.ro2.unique(),group.ro3.unique())))
        r_used = np.concatenate(list(repeat(r_used, 1000)))
        oligo_index = group.index.tolist()
        np.random.shuffle(oligo_index)
        oligo_index = oligo_index[:per_tid]
        base_idx = 0
        c = Counter()
        for i, idx in enumerate(oligo_index):
            ro1_seq = ''
            ro2_seq = ''
            ro3_seq = ''
            oligo = ''
            ro1=''
            ro2=''
            ro3=''
#             try:
            ro1_seq = readouts[r_used[base_idx]]
            ro1 = r_used[base_idx]
        
            ro2_seq = readouts[r_used[base_idx+1]]
            ro2 = r_used[base_idx+1]
            
            ro3_seq = readouts[r_used[base_idx+2]]
            ro3 = r_used[base_idx+2]
            
            c.update([ro1, ro2, ro3])
            row = group.loc[idx]
            rand = np.random.randint(0, high=2)
            if (ro1 not in r_used) or (ro2 not in r_used) or (ro3 not in r_used):
                print(row)
            if rand:
                oligo = row.pleft+'A'+ro1_seq+ro2_seq+row.encodingRegion+'A'+ro3_seq+row.pright

#                     row.set_value(idx, 'oligo', row.pleft+row.ro1+row.ro2+'A'+row.encodingRegion+'A'+row.ro3+row.pright)
            else:
                oligo = row.pleft+'A'+ro1_seq+row.encodingRegion+'A'+ro2_seq+ro3_seq+row.pright
            if (len(row.encodingRegion) != 30) or (len(ro1_seq) != 20):

                print(len(row.encodingRegion), len(ro1_seq))
#                     row.set_value(idx, 'oligo', row.pleft+row.ro1+'A'+row.encodingRegion+'A'+row.ro2+row.ro3+row.pright)
            header = ">"+row.gene+sep+row.tid+sep+str(row.start)+sep+ro1+sep+ro2+sep+ro3+sep+row.experiment+'\n'
            f.write(header)
            f.write(oligo+'\n')
            base_idx += 3
#             except Exception as e:
#                 print(e)
#                 continue
        counters[name] = c
        if len(c.keys())>4:
            print(name)
    f.close()
    return new_df, fa_out, counters

In [133]:
# update spreadsheet with Probe design data
# Sequences, FPKM, number of probes
df = parse_merfish_oligos('/bigstore/GeneralStorage/Rob/merfish/MERFISH_analysis-master/mouse/Cornea_Wound/Cornea_Wound_oligos.fasta')[0]
from collections import Counter
counts = Counter(df.gene)
num_oligos = expression_worksheet.range('E2:E189')
gnames = expression_worksheet.range('A2:A189')
for k, v in counts.items():
    idx = [i for i, g in enumerate(gnames) if g.value==k][0]
    num_oligos[idx].value = v
expression_worksheet.update_cells(num_oligos)

{'spreadsheetId': '11MMvEY4s8oUIGf5kyoA4QOKs-Br2myyNooNmSO7-nEU',
 'updatedCells': 188,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!E2:E189',
 'updatedRows': 188}

In [45]:
gene_list = []
for i in range(len(gene_annotations)):
    gene_list.append(gene_annotations[i][0])